<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/11-Flask/B-Create_API_call_and_Connecting_to_MySQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating of a Flask application showing Citibike stations


In this segment we will create a basic app, where we will connect to the Citibike database, and display the list of stations. 



In [2]:
!pip install -U -q PyMySQL sqlalchemy sql_magic
!pip install -U -q flask # ==0.12.2
!pip install -U -q flask-ngrok

     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 1.6 MB 14.0 MB/s 
     |████████████████████████████████| 1.6 MB 44.7 MB/s 
     |████████████████████████████████| 120 kB 53.9 MB/s 
     |████████████████████████████████| 82 kB 424 kB/s 
     |████████████████████████████████| 101 kB 8.9 MB/s 
     |████████████████████████████████| 96 kB 5.9 MB/s 
     |████████████████████████████████| 232 kB 44.4 MB/s 
     |████████████████████████████████| 133 kB 77.2 MB/s 


## A Refresher 

Let's remember first how to get data from the database.

In [2]:
# This code creates a connection to the database
from sqlalchemy import create_engine

conn_string = 'mysql+pymysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    db = 'citibike_fall2017',
    password = 'dwdstudent2015',
    encoding = 'utf8mb4')

engine = create_engine(conn_string)

In [ ]:
sql = "SELECT DISTINCT id, name, capacity, lat, lon  FROM status_fall2017 LIMIT 5"

with engine.connect() as connection:
  stations = connection.execute(sql).mappings().all()

for s in stations:
    print(s)
    print("-----")

## Creating an API endpoint

#### Now let's start our server:

A few things to notice.

First, notice the `@app.route('/citibike_api',  methods=['GET'])` command. This part specifies that our API endpoint will be accessible under the `http://our_web_server_address/citibike_api`.


Then, we define the function `def citibike_stations():` that will create the response of that API call. What the function returns is what the API call will return back.

Notice that insider the `citibike_stations` function, we connect to the database, and issue an SQL query to the database.

Finally, we get back the results of the query, we put the results in a Python dictionary, and we use the `jsonify` function to convert our dictionary to JSON and return it as the API result.


In [4]:
from flask import Flask, render_template, jsonify
from flask_ngrok import run_with_ngrok
from sqlalchemy import create_engine
 
app = Flask(__name__)
run_with_ngrok(app)
 
@app.route('/citibike_api',  methods=['GET'])
def citibike_stations():
  
  # Setup a connection to the database
  conn_string = 'mysql+pymysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
      host = 'db.ipeirotis.org', 
      user = 'student',
      db = 'citibike_fall2017',
      password = 'dwdstudent2015',
      encoding = 'utf8mb4')
  engine = create_engine(conn_string)

  # Connect to the database, execute the query, and get back the results
  sql = "SELECT DISTINCT id, name, capacity, lat, lon  FROM status_fall2017" 
  with engine.connect() as connection:  
    rows = connection.execute(sql).mappings().all()
    
  # Create the response. We will put the retrieved data as a list of
  # dictionaries, under the key "stations".
  results = {"stations": []}
  for r in rows:
    results["stations"].append(dict(r))

  # We JSON-ify our dictionary and return it as the API response
  return jsonify(results)
 
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://3067-34-86-135-221.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [09/Sep/2022 13:45:43] "GET /citibike_api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Sep/2022 13:45:43] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/Sep/2022 13:45:44] "GET /citibike_api HTTP/1.1" 200 -
